#### Importing libraries

In [1]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import sqlite3
from urllib.parse import urlparse, parse_qs

#### Load Data

In [3]:
df = pd.read_csv('../data/news_articles.csv')
df.head()

,title,link,time
0,"Ohio Rejects Issue 1, Constitutional Change In...",{'https://news.google.com/topics/articles/CBMi...,2023-08-09T03:17:58Z
1,Ohio special election and what it could mean f...,{'https://news.google.com/topics/articles/CCAi...,2023-08-09T00:00:01Z
2,Editorial: Ohio at a crossroad | The Blade,{'https://news.google.com/topics/articles/CBMi...,2023-08-08T04:02:58Z
3,Commentary: Ohio's GOP just learned voters are...,{'https://news.google.com/topics/articles/CBMi...,2023-08-09T02:27:00Z
4,Judge schedules Friday hearing on protective o...,{'https://news.google.com/topics/articles/CBMi...,2023-08-08T23:56:00Z


#### Change column name "title" to "headlines"

In [4]:
df.rename({'title':'headlines','time':'date_time'},axis = 1, inplace = True)
df.head()

,headlines,link,date_time
0,"Ohio Rejects Issue 1, Constitutional Change In...",{'https://news.google.com/topics/articles/CBMi...,2023-08-09T03:17:58Z
1,Ohio special election and what it could mean f...,{'https://news.google.com/topics/articles/CCAi...,2023-08-09T00:00:01Z
2,Editorial: Ohio at a crossroad | The Blade,{'https://news.google.com/topics/articles/CBMi...,2023-08-08T04:02:58Z
3,Commentary: Ohio's GOP just learned voters are...,{'https://news.google.com/topics/articles/CBMi...,2023-08-09T02:27:00Z
4,Judge schedules Friday hearing on protective o...,{'https://news.google.com/topics/articles/CBMi...,2023-08-08T23:56:00Z


#### Check for null values

In [5]:
df.isna().sum()

headlines    0
link         0
date_time    0
dtype: int64

#### Check for duplicates

In [6]:
df.duplicated().sum()

0

#### Format date_time column to standard format

In [7]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d')
df.head()

,headlines,link,date_time
0,"Ohio Rejects Issue 1, Constitutional Change In...",{'https://news.google.com/topics/articles/CBMi...,2023-08-09 03:17:58+00:00
1,Ohio special election and what it could mean f...,{'https://news.google.com/topics/articles/CCAi...,2023-08-09 00:00:01+00:00
2,Editorial: Ohio at a crossroad | The Blade,{'https://news.google.com/topics/articles/CBMi...,2023-08-08 04:02:58+00:00
3,Commentary: Ohio's GOP just learned voters are...,{'https://news.google.com/topics/articles/CBMi...,2023-08-09 02:27:00+00:00
4,Judge schedules Friday hearing on protective o...,{'https://news.google.com/topics/articles/CBMi...,2023-08-08 23:56:00+00:00


#### Format URL

In [11]:
def format_url(link):
    parsed_url = urlparse(link)
    path_components = parsed_url.path.split('/')
    article_id = path_components[-1]
    query_params = parse_qs(parsed_url.query)
    hl_param = query_params.get('hl', [''])[0]
    gl_param = query_params.get('gl', [''])[0]
    ceid_param = query_params.get('ceid', [''])[0]
    
    formatted_info = {
        'article_id': article_id,
        'language': hl_param,
        'region': gl_param,
        'ceid': ceid_param
    }
    return formatted_info

In [12]:
df[['article_id', 'language', 'region', 'ceid']] = df['link'].apply(format_url).apply(pd.Series)
df.head()

,headlines,link,date_time,article_id,language,region,ceid
0,"Ohio Rejects Issue 1, Constitutional Change In...",{'https://news.google.com/topics/articles/CBMi...,2023-08-09 03:17:58+00:00,CBMiSGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjMvMD...,en-US,US,US:en'}
1,Ohio special election and what it could mean f...,{'https://news.google.com/topics/articles/CCAi...,2023-08-09 00:00:01+00:00,CCAiCzNGMzZPVk1oNWljmAEB,en-US,US,US:en'}
2,Editorial: Ohio at a crossroad | The Blade,{'https://news.google.com/topics/articles/CBMi...,2023-08-08 04:02:58+00:00,CBMiaWh0dHBzOi8vd3d3LnRvbGVkb2JsYWRlLmNvbS9vcG...,en-US,US,US:en'}
3,Commentary: Ohio's GOP just learned voters are...,{'https://news.google.com/topics/articles/CBMi...,2023-08-09 02:27:00+00:00,CBMiSmh0dHBzOi8vd3d3Lnd2eHUub3JnL3BvbGl0aWNzLz...,en-US,US,US:en'}
4,Judge schedules Friday hearing on protective o...,{'https://news.google.com/topics/articles/CBMi...,2023-08-08 23:56:00+00:00,CBMiUWh0dHBzOi8vd3d3LmNubi5jb20vMjAyMy8wOC8wOC...,en-US,US,US:en'}


In [13]:
conn = sqlite3.connect('../data/etl.db')

In [14]:
table_name = 'news'

In [15]:
df.to_sql(table_name, conn, if_exists='append', index=False)